In [1]:
!pip install transformers xgboost

In [2]:
from transformers import AutoTokenizer, TFAutoModel, AutoConfig, pipeline, AutoModel
import re
import io
import pandas as pd
import tensorflow as tf
import numpy as np
from sklearn.metrics import f1_score, accuracy_score,precision_score,recall_score
import os
from google.colab import drive
seed =45
np.random.seed(seed)
tf.random.set_seed(seed)
tpu_strategy = tf.distribute.experimental.TPUStrategy(tf.distribute.cluster_resolver.TPUClusterResolver.connect())
drive.mount('/gdrive')

INFO:tensorflow:Deallocate tpu buffers before initializing tpu system.


INFO:tensorflow:Deallocate tpu buffers before initializing tpu system.


INFO:tensorflow:Initializing the TPU system: grpc://10.43.228.58:8470


INFO:tensorflow:Initializing the TPU system: grpc://10.43.228.58:8470


INFO:tensorflow:Finished initializing TPU system.


...

INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).


In [3]:
model_type="UBC-NLP/ARBERT"
language="Ara"
data_dir="/content"
train_file="ara_train.csv"
dev_file="ara_dev.csv"
model_dir="/content"
model_file=f"{language}-Mono-Model.h5"
train_path=os.path.join(data_dir,train_file)
dev_path=os.path.join(data_dir,dev_file)
model_path=os.path.join(model_dir,model_file)

In [4]:
train_data=pd.read_csv(train_path)
valid_data=pd.read_csv(dev_path)

In [5]:
def balanced_batch(data):
  harmful=data.loc[data["class_label"]==1]
  norm_count=int(len(harmful)*1.5)
  normal=data.loc[data["class_label"]==0].sample(n=norm_count)
  data=harmful.append(normal).sample(frac=1).reset_index(drop=True)
  return data

train_data=balanced_batch(train_data)

In [6]:
def gen_data(data,tokenizer_type=model_type):
    data_dict={}
    data_dict["harmful_label"]=tf.convert_to_tensor(data["class_label"].values.astype(int))
    data_dict["text"]=data["tweet_text"].values.astype("str")
    tokenizer = AutoTokenizer.from_pretrained(tokenizer_type)
    input_ids, input_masks = [],[]
    for sentence in data_dict["text"]:
        inputs = tokenizer.encode_plus(sentence, add_special_tokens=True,max_length=326,truncation=True, padding='max_length',return_attention_mask=True, return_token_type_ids=True)
        input_ids.append(inputs['input_ids'])
        input_masks.append(inputs['attention_mask'])
    data_dict["input_ids"]=tf.convert_to_tensor(input_ids) 
    data_dict["input_masks"]=tf.convert_to_tensor(input_masks)  
    return data_dict

train=gen_data(train_data)
valid=gen_data(valid_data)

In [7]:
# def create_mono_model(transformer):
#     input_ids= tf.keras.layers.Input(shape=(326,), dtype='int32')
#     input_masks = tf.keras.layers.Input(shape=(326,), dtype='int32')
#     embedding_layer=transformer(input_ids, attention_mask=input_masks)[0]
#     #embedding_layer= tf.keras.layers.SpatialDropout1D(0.2)(embedding_layer)
    
#     harmful=tf.keras.layers.Bidirectional(tf.keras.layers.GRU(64, return_sequences=True, dropout=0.2))(embedding_layer)
#     harmful=tf.keras.layers.Bidirectional(tf.keras.layers.GRU(32, return_sequences=True, dropout=0.1))(harmful)
#     harmful = tf.keras.layers.Conv1D(32, kernel_size = 3, padding = "valid", kernel_initializer = "glorot_uniform")(harmful)
#     avg_pool_harmful = tf.keras.layers.GlobalAveragePooling1D()(harmful)
#     max_pool_harmful = tf.keras.layers.GlobalMaxPooling1D()(harmful)
#     harmful = tf.keras.layers.concatenate([avg_pool_harmful, max_pool_harmful])
#     harmful = tf.keras.layers.Dropout(0.1)(harmful)
#     output_harmful=tf.keras.layers.Dense(1, activation="sigmoid",name="output_harmful")(harmful)

#     model = tf.keras.Model(inputs=[input_ids, input_masks], outputs =output_harmful)
    
#     return model

In [8]:
def create_mono_model(transformer):
    input_ids= tf.keras.layers.Input(shape=(326,), dtype='int32')
    input_masks = tf.keras.layers.Input(shape=(326,), dtype='int32')
    embedding_layer=transformer(input_ids, attention_mask=input_masks)[0]
    #embedding_layer= tf.keras.layers.SpatialDropout1D(0.2)(embedding_layer)
    
    harmful=tf.keras.layers.Bidirectional(tf.keras.layers.GRU(256, return_sequences=True, dropout=0.3))(embedding_layer)
    harmful=tf.keras.layers.Bidirectional(tf.keras.layers.GRU(128, return_sequences=True, dropout=0.3))(harmful)
    harmful=tf.keras.layers.Bidirectional(tf.keras.layers.GRU(64, return_sequences=True, dropout=0.3))(harmful)
    harmful = tf.keras.layers.Conv1D(64, kernel_size = 3, padding = "valid", kernel_initializer = "glorot_uniform")(harmful)
    harmful = tf.keras.layers.Dropout(0.2)(harmful)
    avg_pool_harmful = tf.keras.layers.GlobalAveragePooling1D()(harmful)
    max_pool_harmful = tf.keras.layers.GlobalMaxPooling1D()(harmful)
    harmful = tf.keras.layers.concatenate([avg_pool_harmful, max_pool_harmful])
    harmful = tf.keras.layers.Dropout(0.1)(harmful)
    output_harmful=tf.keras.layers.Dense(1, activation="sigmoid",name="output_harmful")(harmful)

    model = tf.keras.Model(inputs=[input_ids, input_masks], outputs =output_harmful)
    
    return model

In [9]:
def build_mono_model(compile=False,show_summary=False,pretrained_model=model_type):
    with tpu_strategy.scope():
      config =AutoConfig.from_pretrained(pretrained_model,dropout=0.2,seed=3,attention_dropout=0.2,output_hidden_states = True)
      transformer= TFAutoModel.from_pretrained(pretrained_model,config=config)
      model =create_mono_model(transformer)
      if compile : 
          model.compile(loss={"output_harmful":"binary_crossentropy"},
                      optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4), 
                      metrics={"output_harmful":"accuracy"}
                     )
      if show_summary:
        model.summary()
      return model

model=build_mono_model()

def train_model(model,train,valid,batch_size=64,epochs_frozen=4,epochs_unfrozen=12):
      
    # checkpoint_filepath = data_path

    # model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    #     checkpoint_filepath, 
    #     monitor='val_loss', 
    #     save_weights_only=True, 
    #     save_best_only=True, 
    #     save_freq='epoch', 
    #     options=tf.train.CheckpointOptions(experimental_io_device='/job:localhost'))
    
    for layer in model.layers[:3]:
        layer.trainable = False
   
    with tpu_strategy.scope():
      model.compile(loss={"output_harmful":"binary_crossentropy"},
                      optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4), 
                      metrics={"output_harmful":"accuracy"}
                     )
        
    model.fit([train["input_ids"], train["input_masks"]],train["harmful_label"],batch_size=batch_size,epochs=epochs_frozen,validation_data=([valid["input_ids"],valid["input_masks"]],valid["harmful_label"]))#, callbacks=[model_checkpoint_callback])
   
    for layer in model.layers[:3]:
        layer.trainable = True

    with tpu_strategy.scope():
      model.compile(loss={"output_harmful":"binary_crossentropy"}, 
                      optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5), 
                      metrics={"output_harmful":"accuracy"})
    
    model.fit([train["input_ids"], train["input_masks"]],train["harmful_label"],batch_size=batch_size,epochs=epochs_unfrozen,validation_data=([valid["input_ids"],valid["input_masks"]],valid["harmful_label"]))#, callbacks=[model_checkpoint_callback])

train_model(model,train,valid)

All model checkpoint layers were used when initializing TFBertModel.

All the layers of TFBertModel were initialized from the model checkpoint at UBC-NLP/ARBERT.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


Epoch 1/4
29/29 [==============================] - 111s 2s/step - loss: 0.6480 - accuracy: 0.6602 - val_loss: 0.3847 - val_accuracy: 0.8300
Epoch 2/4
29/29 [==============================] - 10s 348ms/step - loss: 0.4700 - accuracy: 0.7729 - val_loss: 0.3355 - val_accuracy: 0.8511
Epoch 3/4
29/29 [==============================] - 10s 346ms/step - loss: 0.4063 - accuracy: 0.8173 - val_loss: 0.3034 - val_accuracy: 0.8602
Epoch 4/4
29/29 [==============================] - 10s 347ms/step - loss: 0.3732 - accuracy: 0.8331 - val_loss: 0.2912 - val_accuracy: 0.8692
Epoch 1/12


29/29 [==============================] - 236s 4s/step - loss: 0.4018 - accuracy: 0.8260 - val_loss: 0.2593 - val_accuracy: 0.8903
Epoch 2/12
29/29 [==============================] - 17s 581ms/step - loss: 0.2432 - accuracy: 0.8997 - val_loss: 0.2683 - val_accuracy: 0.8893
Epoch 3/12
29/29 [==============================] - 17s 583ms/step - loss: 0.1560 - accuracy: 0.9507 - val_loss: 0.2547 - val_accuracy: 0.9044
Epoch 4/12
29/29 [==============================] - 17s 594ms/step - loss: 0.0922 - accuracy: 0.9772 - val_loss: 0.2685 - val_accuracy: 0.8944
Epoch 5/12
29/29 [==============================] - 17s 582ms/step - loss: 0.0568 - accuracy: 0.9881 - val_loss: 0.4125 - val_accuracy: 0.8702
Epoch 6/12
29/29 [==============================] - 17s 583ms/step - loss: 0.0403 - accuracy: 0.9902 - val_loss: 0.3251 - val_accuracy: 0.8984
Epoch 7/12
29/29 [==============================] - 17s 586ms/step - loss: 0.0264 - accuracy: 0.9930 - val_loss: 0.3328 - val_accuracy: 0.8964
Epoch 8/12
2

In [10]:
def report_gen(predictions,labels,avg="binary"):
    report={
    "F1_macro":f1_score(predictions,labels,average=avg),
    "Accuracy":accuracy_score(predictions,labels),
    "Precision_macro":precision_score(predictions,labels),
    "Recall_macro":recall_score(predictions,labels)
    }
    return report

def eval_task_harmful(predictions_harmful,labels_harmful):
  predictions_harmful=[int(i>0.7) for i in predictions_harmful]
  return report_gen(predictions_harmful,labels_harmful),predictions_harmful

def eval_model(model,valid):
    predictions_harmful=model.predict([valid["input_ids"],valid["input_masks"]])
    report_harmful,predictions_harmful=eval_task_harmful(predictions_harmful,valid["harmful_label"])
    print(report_harmful)
    return predictions_harmful

predictions=eval_model(model,valid)

{'F1_macro': 0.6968085106382979, 'Accuracy': 0.8853118712273642, 'Precision_macro': 0.6931216931216931, 'Recall_macro': 0.7005347593582888}


In [11]:
#128 64 64
#{'F1_macro': 0.7364016736401674, 'Accuracy': 0.8732394366197183, 'Precision_macro': 0.9312169312169312, 'Recall_macro': 0.6089965397923875}
#256 128 64
#{'F1_macro': 0.7505720823798626, 'Accuracy': 0.8903420523138833, 'Precision_macro': 0.8677248677248677, 'Recall_macro': 0.6612903225806451}
#256 128 64 32
#{'F1_macro': 0.7443037974683544, 'Accuracy': 0.8983903420523138, 'Precision_macro': 0.7777777777777778, 'Recall_macro': 0.7135922330097088}
#256 128 64 64
#{'F1_macro': 0.7623762376237623, 'Accuracy': 0.903420523138833, 'Precision_macro': 0.8148148148148148, 'Recall_macro': 0.7162790697674418}
#256 128 64 64
#{'F1_macro': 0.7535545023696683, 'Accuracy': 0.8953722334004024, 'Precision_macro': 0.8412698412698413, 'Recall_macro': 0.6824034334763949}
#256 128 64 (0.2) 64
#{'F1_macro': 0.7611940298507462, 'Accuracy': 0.903420523138833, 'Precision_macro': 0.8095238095238095, 'Recall_macro': 0.7183098591549296}
#12 epochs
#{'F1_macro': 0.7436974789915967, 'Accuracy': 0.8772635814889336, 'Precision_macro': 0.9365079365079365, 'Recall_macro': 0.6167247386759582}

#{'F1_macro': 0.7627906976744185, 'Accuracy': 0.89738430583501, 'Precision_macro': 0.8677248677248677, 'Recall_macro': 0.6804979253112033}
# model.save_weights(model_path)

In [12]:
#pd.DataFrame(predictions).to_csv("/content/predictions.csv",index=False, header=None)